In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
#!unzip drive/MyDrive/ColabNotebooks/7.0.NOTEBOOK_UC/CV/data/archive.zip

In [ ]:
#!pip install imageo

In [ ]:
!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

In [ ]:
#!unzip kagglecatsanddogs_5340.zip

In [1]:
!ls PetImages

Cat Dog


In [120]:
%%writefile ImageClassification.py
import os
import argparse


# ARGS
parser = argparse.ArgumentParser()
parser.add_argument("--num_epochs", type=int, help="number of epochs")
parser.add_argument("--dir_train", type=str, help="the directory for training")
parser.add_argument("--learning_rate", type=float, help="learning_rate")
parser.add_argument("--image_size", type=int, help="image_size")
parser.add_argument("--checkpoint_path", type=str, help="checkpoint_path")
parser.add_argument("--batch_size", type=int, help="batch_size")
parser.add_argument("--data_aug", type=bool, help="data_aug")
parser.add_argument("--num_classes", type=int, help="num_classes")

args = parser.parse_args()

dir_train=args.dir_train
num_epochs=args.num_epochs
learning_rate=args.learning_rate
image_size=(args.image_size, args.image_size)
checkpoint_path=args.checkpoint_path
batch_size=args.batch_size
data_aug =args.data_aug
num_classes=args.num_classes

    
# UTILS
import tensorflow as tf 
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing import image

import tensorflow.keras.layers as tfl
from tensorflow.keras import layers

from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation




def read_data_from_dir(directory,
                       image_size,
                       batch_size,
                       work_on_sample=False
                      ):

    train_dataset = image_dataset_from_directory(directory,
                                                 shuffle=True,
                                                 batch_size=batch_size,
                                                 image_size=image_size,
                                                 validation_split=0.2,
                                                 subset='training',
                                                 seed=42)

    val_dataset = image_dataset_from_directory(directory,
                                                 shuffle=True,
                                                 batch_size=batch_size,
                                                 image_size=image_size,
                                                 validation_split=0.2,
                                                 subset='validation',
                                                 seed=42)
    if work_on_sample:
        train_dataset=train_dataset.take(1)
        val_dataset= val_dataset.take(1)   
        
    
    
    return train_dataset, val_dataset

def data_augmenter():
    '''
    Create a Sequential model composed of 2 layers
    Returns:
        tf.keras.Sequential
    '''
    data_augmentation = tf.keras.Sequential() # define the sequential
    data_augmentation.add(RandomFlip('horizontal'))
    data_augmentation.add(RandomRotation(0.2))
    
    return data_augmentation


def image_classification(image_shape,
                         base_model,
                         data_augmentation,
                         preprocess_input,
                         learning_rate,
                         num_classes,
                         data_aug
                        ):
    ''' Define a tf.keras model for binary classification out of the MobileNetV2 model
    Arguments:
        image_shape -- Image width and height
        data_augmentation -- data augmentation function
    Returns:
    Returns:
        tf.keras.model
    '''
    input_shape = image_shape
    #+ (3,)
    base_model.trainable = True
    
    # Fine-tune from this layer onwards
    fine_tune_at = 120

    # Freeze all the layers before the `fine_tune_at` layer
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    # create the input layer (Same as the imageNetv2 input size)
    inputs = tf.keras.Input(shape=input_shape) 
    
    if data_aug:
        x = data_augmentation(inputs)
    else:
        x = inputs
    
    # data preprocessing using the same weights the model was trained on
    x = preprocess_input(x)
    #x = preprocess_input(inputs) 
    
    # set training to False to avoid keeping track of statistics in the batch norm layer
    x = base_model(x) 
    
    x = tfl.GlobalAveragePooling2D()(x) 
    
    x = tfl.Dropout(0.2)(x)
        
    # use a prediction layer with one neuron (as a binary classifier only needs one)
    if num_classes == 2:
        activation = "sigmoid"
        loss_function=tf.keras.losses.BinaryCrossentropy(from_logits=True)
        units = 1
    else:
        activation = "softmax"
        loss_function=tf.keras.losses.CategoricalCrossentropy(from_logits=True)
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    
    model = tf.keras.Model(inputs, outputs)
    
    # compile
    optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
    metrics=['accuracy']


    model.compile(loss = loss_function,
                  optimizer = optimizer,
                  metrics = metrics)

    return model

# MAIN
def main():

    train_data , val_data= read_data_from_dir(dir_train, 
                                              image_size, 
                                              batch_size, 
                                              work_on_sample=True)
    
    image_shape = image_size + (3,)
    base_model = tf.keras.applications.MobileNetV2(input_shape=image_shape,
                                                   include_top=False,
                                                   weights='imagenet')

    preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
    decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions
    data_augmentation = data_augmenter()
    
    model = image_classification(image_shape, 
                                 base_model, 
                                 data_augmentation,
                                 preprocess_input,
                                 learning_rate,
                                 num_classes,
                                 data_aug
                                )
    
    checkpoint_dir = os.path.dirname(checkpoint_path)

    # Create a callback that saves the model's weights
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1)


    history_fine = model.fit(train_data,
                             epochs=num_epochs,
                             validation_data=val_data,
                             callbacks=[cp_callback])
    
    
    print("I am done training")


    

if __name__=="__main__":
    main()



Overwriting ImageClassification.py


In [2]:
!python ImageClassification.py --dir_train="PetImages" --num_epochs=1 --batch_size=32 --learning_rate=0.001 \
                               --image_size=160 --checkpoint_path="model/cp.ckpt" --data_aug=True --num_classes=2


2023-03-16 16:46:17.947805: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Found 23410 files belonging to 2 classes.
Using 18728 files for training.
2023-03-16 16:46:25.345754: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Found 23410 files belonging to 2 classes.
Using 4682 files for validation.
/Users/kindi/opt/anaconda3/lib/python3.8/site-packages/keras/backend.py:5673: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argumen

In [3]:
!ls model

checkpoint                  cp.ckpt.index
cp.ckpt.data-00000-of-00001


# INFERENCE

In [126]:
%%writefile InferenceForImageClass.py
import os
import argparse


# ARGS
parser = argparse.ArgumentParser()
parser.add_argument("--input_image", type=str, help="input_image")
parser.add_argument("--learning_rate", type=float, help="learning_rate")
parser.add_argument("--image_size", type=int, help="image_size")
parser.add_argument("--checkpoint_path", type=str, help="checkpoint_path")
parser.add_argument("--data_aug", type=bool, help="data_aug")
parser.add_argument("--num_classes", type=int, help="num_classes")

args = parser.parse_args()

learning_rate=args.learning_rate
image_size=(args.image_size, args.image_size)
checkpoint_path=args.checkpoint_path
data_aug =args.data_aug
num_classes=args.num_classes
input_image = args.input_image


# UTILS
import tensorflow as tf 
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing import image

import tensorflow.keras.layers as tfl
from tensorflow.keras import layers

from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation



def data_augmenter():
    '''
    Create a Sequential model composed of 2 layers
    Returns:
        tf.keras.Sequential
    '''
    data_augmentation = tf.keras.Sequential() # define the sequential
    data_augmentation.add(RandomFlip('horizontal'))
    data_augmentation.add(RandomRotation(0.2))
    
    return data_augmentation


def image_classification(image_shape,
                         base_model,
                         data_augmentation,
                         preprocess_input,
                         learning_rate,
                         num_classes,
                         data_aug
                        ):
    ''' Define a tf.keras model for binary classification out of the MobileNetV2 model
    Arguments:
        image_shape -- Image width and height
        data_augmentation -- data augmentation function
    Returns:
    Returns:
        tf.keras.model
    '''
    input_shape = image_shape
    #+ (3,)
    base_model.trainable = True
    
    # Fine-tune from this layer onwards
    fine_tune_at = 120

    # Freeze all the layers before the `fine_tune_at` layer
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    # create the input layer (Same as the imageNetv2 input size)
    inputs = tf.keras.Input(shape=input_shape) 
    
    if data_aug:
        x = data_augmentation(inputs)
    else:
        x = inputs
    
    # data preprocessing using the same weights the model was trained on
    x = preprocess_input(x)
    #x = preprocess_input(inputs) 
    
    # set training to False to avoid keeping track of statistics in the batch norm layer
    x = base_model(x) 
    
    x = tfl.GlobalAveragePooling2D()(x) 
    
    x = tfl.Dropout(0.2)(x)
        
    # use a prediction layer with one neuron (as a binary classifier only needs one)
    if num_classes == 2:
        activation = "sigmoid"
        loss_function=tf.keras.losses.BinaryCrossentropy(from_logits=True)
        units = 1
    else:
        activation = "softmax"
        loss_function=tf.keras.losses.CategoricalCrossentropy(from_logits=True)
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    
    model = tf.keras.Model(inputs, outputs)
    
    # compile
    optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
    metrics=['accuracy']


    model.compile(loss = loss_function,
                  optimizer = optimizer,
                  metrics = metrics)

    return model






image_shape = image_size + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=image_shape,
                                                include_top=False,
                                                weights='imagenet')

    
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions
data_augmentation = data_augmenter()
    
model = image_classification(image_shape, 
                             base_model, 
                             data_augmentation,
                             preprocess_input,
                             learning_rate,
                             num_classes,
                             data_aug
                            )



def main():
    
    
    img = image.load_img(input_image, target_size=image_size)
    
    img_array = image.img_to_array(img)
    
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    
    model.load_weights(checkpoint_path)
    
    predictions = model.predict(img_array)
    
    score = float(predictions[0])
    if score ==1:
        print("This image is a dog")
    else:
        print("This image is a cat")
        
    
if __name__=="__main__":
    main()
    

Overwriting InferenceForImageClass.py


In [5]:
!python InferenceForImageClass.py --input_image="PetImages/Dog/6.jpg" --learning_rate=0.001 \
                               --image_size=160 --checkpoint_path="model/cp.ckpt" --data_aug=False \
                               --num_classes=2


2023-03-16 16:47:33.599666: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-16 16:47:36.609915: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
1/1 [==============================] - 1s 563ms/step
This image is a cat


# DEPLOYMENT API

#### FLASK API
#### FAST API
#### DJANGO REST API
#### MLFLOW
#### GRADIO
#### STREAMLIT
#### TENSORFLOW LIKE 


The objectif is to have a web like solution to show case the ML Model in production